# Regression Week 2: Multiple Regression (Interpretation)

The goal of this first notebook is to explore multiple regression and feature engineering with existing graphlab functions.

In this notebook you will use data on house sales in King County to predict prices using multiple regression. You will:
* Use SFrames to do some feature engineering
* Use built-in graphlab functions to compute the regression weights (coefficients/parameters)
* Given the regression weights, predictors and outcome write a function to compute the Residual Sum of Squares
* Look at coefficients and interpret their meanings
* Evaluate multiple models via RSS

In [27]:
import pandas as pd
import numpy as np
import sklearn
import seaborn as sn

# Load in house sales data

Dataset is from house sales in King County, the region where the city of Seattle, WA is located.

In [38]:
dtype_dict = {'bathrooms':float, 'waterfront':int, 'sqft_above':int, 'sqft_living15':float, 'grade':int, 'yr_renovated':int, 'price':float, 'bedrooms':float, 'zipcode':str, 'long':float, 'sqft_lot15':float, 'sqft_living':float, 'floors':str, 'condition':int, 'lat':float, 'date':str, 'sqft_basement':int, 'yr_built':int, 'id':str, 'sqft_lot':int, 'view':int}
sales = pd.read_csv("kc_house_data.csv",dtype=dtype_dict)

# Split data into training and testing.

In [39]:
from sklearn.cross_validation import train_test_split
train, test = train_test_split(sales, test_size=0.2, random_state=0)

# Learning a multiple regression model

Use linear_model.LinearRegression to train

In [40]:
def statsprinter(model):
    print "Intercept: ", model.intercept_
    print "Coefficients: ", model.coef_

In [41]:
example_features = ['sqft_living', 'bedrooms', 'bathrooms']
simple_train = train[example_features]
simple_model = sklearn.linear_model.LinearRegression()
simple_model.fit(simple_train, train['price'])

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

Now that we have fitted the model we can extract the regression weights (coefficients) as an SFrame as follows:

In [42]:
statsprinter(simple_model)

Intercept:  67512.0151381
Coefficients:  [   313.17055038 -56754.66651422   6887.71910816]


# Making Predictions

In [43]:
simple_predictions = example_model.predict(test[example_features])
print simple_predictions[0]

412168.147818


# Compute RSS

Now that we can make predictions given the model, let's write a function to compute the RSS of the model. Complete the function below to calculate RSS given the model, data, and the outcome.

In [44]:
def get_residual_sum_of_squares(model, data, outcome):
    # First get the predictions
    predictions = model.predict(data)
    # Then compute the residuals/errors
    errors = predictions - outcome
    # Then square and add them up
    RSS = 0
    for item in errors:
        RSS += item ** 2
    return(RSS)    

Test your function by computing the RSS on TEST data for the example model:

In [45]:
rss_example_train = get_residual_sum_of_squares(simple_model, test[example_features], test['price'])
print rss_example_train

2.59213572106e+14


# Create some new features

Although we often think of multiple regression as including multiple different features (e.g. # of bedrooms, squarefeet, and # of bathrooms) but we can also consider transformations of existing features e.g. the log of the squarefeet or even "interaction" features such as the product of bedrooms and bathrooms.

You will use the logarithm function to create a new feature. so first you should import it from the math library.

In [46]:
from math import log

Next create the following 4 new features as column in both TEST and TRAIN data:
* bedrooms_squared = bedrooms\*bedrooms
* bed_bath_rooms = bedrooms\*bathrooms
* log_sqft_living = log(sqft_living)
* lat_plus_long = lat + long 
As an example here's the first one:

In [47]:
train['bedrooms_squared'] = train['bedrooms'].apply(lambda x: x**2)
test['bedrooms_squared'] = test['bedrooms'].apply(lambda x: x**2)

/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


In [65]:
# create the remaining 3 features in both TEST and TRAIN data
train['bedXbath'] = train['bedrooms'] * train['bathrooms']
test['bedXbath'] = test['bedrooms'] * test['bathrooms']
train['logsqft'] = train['sqft_living'].apply(lambda x: log(x))
test['logsqft'] = test['sqft_living'].apply(lambda x: log(x))
train['lat_plus_long'] = train['lat'] + train['long']
test['lat_plus_long'] = test['lat'] + test['long']

/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()
/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas

* Squaring bedrooms will increase the separation between not many bedrooms (e.g. 1) and lots of bedrooms (e.g. 4) since 1^2 = 1 but 4^2 = 16. Consequently this feature will mostly affect houses with many bedrooms.
* bedrooms times bathrooms gives what's called an "interaction" feature. It is large when *both* of them are large.
* Taking the log of squarefeet has the effect of bringing large values closer together and spreading out small values.
* Adding latitude to longitude is totally non-sensical but we will do it anyway (you'll see why)

# Learning Multiple Models

Now we will learn the weights for three (nested) models for predicting house prices. The first model will have the fewest features the second model will add one more feature and the third will add a few more:
* Model 1: squarefeet, # bedrooms, # bathrooms, latitude & longitude
* Model 2: add bedrooms\*bathrooms
* Model 3: Add log squarefeet, bedrooms squared, and the (nonsensical) latitude + longitude

In [66]:
model_1_features = ['sqft_living', 'bedrooms', 'bathrooms', 'lat', 'long']
model_2_features = model_1_features + ['bedXbath']
model_3_features = model_2_features + ['bedrooms_squared', 'logsqft', 'lat_plus_long']

Now that you have the features, learn the weights for the three different models for predicting target = 'price' using graphlab.linear_regression.create() and look at the value of the weights/coefficients:

In [67]:
# Learn the three models: (don't forget to set validation_set = None)
model_1 = sklearn.linear_model.LinearRegression()
model_1.fit(train[model_1_features], train['price'])
model_2 = sklearn.linear_model.LinearRegression()
model_2.fit(train[model_2_features], train['price'])
model_3 = sklearn.linear_model.LinearRegression()
model_3.fit(train[model_3_features], train['price'])

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [68]:
# Examine/extract each model's coefficients:
statsprinter(model_1)
statsprinter(model_2)
statsprinter(model_3)

Intercept:  -70870846.2389
Coefficients:  [  3.12942010e+02  -5.30962691e+04   1.47770428e+04   6.53983343e+05
  -3.25707336e+05]
Intercept:  -68606820.3957
Coefficients:  [  3.06819573e+02  -1.04604718e+05  -7.01815289e+04   6.50590952e+05
  -3.09965751e+05   2.49441497e+04]
Intercept:  -62628450.2722
Coefficients:  [  5.37808086e+02   2.78047910e+03   1.01363766e+05   5.30798406e+05
  -4.09655435e+05  -1.81822552e+04   7.24579939e+02  -5.71030023e+05
   1.21142971e+05]


**Quiz Question: What is the sign (positive or negative) for the coefficient/weight for 'bathrooms' in model 1?**

**Quiz Question: What is the sign (positive or negative) for the coefficient/weight for 'bathrooms' in model 2?**

Think about what this means.

# Comparing multiple models

Now that you've learned three models and extracted the model weights we want to evaluate which model is best.

First use your functions from earlier to compute the RSS on TRAINING Data for each of the three models.

In [71]:
# Compute the RSS on TRAINING data for each of the three models and record the values:
print get_residual_sum_of_squares(data=train[model_1_features], model=model_1, outcome=train['price'])
print get_residual_sum_of_squares(data=train[model_2_features], model=model_2, outcome=train['price'])
print get_residual_sum_of_squares(data=train[model_3_features], model=model_3, outcome=train['price'])

9.79843597588e+14
9.7079919973e+14
9.13653644975e+14


**Quiz Question: Which model (1, 2 or 3) has lowest RSS on TRAINING Data?** Is this what you expected?

Now compute the RSS on on TEST data for each of the three models.

In [72]:
# Compute the RSS on TESTING data for each of the three models and record the values:
print get_residual_sum_of_squares(data=test[model_1_features], model=model_1, outcome=test['price'])
print get_residual_sum_of_squares(data=test[model_2_features], model=model_2, outcome=test['price'])
print get_residual_sum_of_squares(data=test[model_3_features], model=model_3, outcome=test['price'])

2.13487129319e+14
2.10778544169e+14
2.03972051918e+14


**Quiz Question: Which model (1, 2 or 3) has lowest RSS on TESTING Data?** Is this what you expected? Think about the features that were added to each model from the previous.